# Exercício 2


## Sobre a base de dados

Neste laboratório você utilizará um dataset contendo informações sobre filmes e cinemas. 

**Collections:**
* comments
* movies
* sessions
* theaters
* users




Veja um exemplo de documento de um dos filmes da collection `movies` e entenda a estrutura/schema:

```python
{
	'_id': ObjectId('573a1390f29313caabcd4135'),
	'awards': {
		'nominations': 0,
		'text': '1 win.',
		'wins': 1
	},
	'cast': ['Charles Kayser', 'John Ott'],
	'countries': ['USA'],
	'directors': ['William K.L. Dickson'],
	'fullplot': 'A stationary camera looks at a large ...',
	'genres': ['Short'],
	'imdb': {
		'id': 5,
		'rating': 6.2,
		'votes': 1189
	},
	'lastupdated': '2015-08-26 00:03:50.133000000',
	'num_mflix_comments': 1,
	'plot': 'Three men hammer on an anvil and pass a bottle of beer around.',
	'rated': 'UNRATED',
	'released': datetime(1893, 5, 9, 0, 0),
	'runtime': 1,
	'title': 'Blacksmith Scene',
	'tomatoes': {
		'lastUpdated': datetime(2015, 6, 28, 18, 34, 9),
		'viewer': {
			'meter': 32,
			'numReviews': 184,
			'rating': 3.0
		},
	},
	'type': 'movie',
	'year': 1893,
}
```



## Usando Python

In [1]:
from pymongo import MongoClient
from pprintpp import pprint
import warnings
warnings.filterwarnings('ignore')
mongoclient = MongoClient('localhost', 27017)
db = mongoclient.mflix
print(db)

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'mflix')


In [2]:
db.movies.find({}).count()

23539

## Questões Exemplo

### [Questão Exemplo A]  Mostre um documento qualquer da coleção movies

In [3]:
result = db.movies.find({}).limit(1)

pprint( list(result) )

[
    {
        '_id': ObjectId('573a1390f29313caabcd4135'),
        'awards': {'nominations': 0, 'text': '1 win.', 'wins': 1},
        'cast': ['Charles Kayser', 'John Ott'],
        'countries': ['USA'],
        'directors': ['William K.L. Dickson'],
        'fullplot': 'A stationary camera looks at a large anvil with a blacksmith behind it and one on either side. The smith in the middle draws a heated metal rod from the fire, places it on the anvil, and all three begin a rhythmic hammering. After several blows, the metal goes back in the fire. One smith pulls out a bottle of beer, and they each take a swig. Then, out comes the glowing metal and the hammering resumes.',
        'genres': ['Short'],
        'imdb': {'id': 5, 'rating': 6.2, 'votes': 1189},
        'lastupdated': '2015-08-26 00:03:50.133000000',
        'num_mflix_comments': 1,
        'plot': 'Three men hammer on an anvil and pass a bottle of beer around.',
        'rated': 'UNRATED',
        'released': datetime.datet

### [Questão Exemplo B]  Encontre os diretores do filme "Inglourious Basterds"

In [4]:
result = db.movies.find({"title": "Inglourious Basterds"}, {"_id":0,"directors": 1})

list(result)

[{'directors': ['Quentin Tarantino', 'Eli Roth']}]

### [Questão 1] Mostre o ano de lançamento de todos os filme que começam com "One Night"

In [5]:
result = db.movies.find({'title':{'$regex':'^One Night'}},
                        {'title':1,'year':1, '_id':0}
                       )
list(result)

[{'title': 'One Night of Love', 'year': 1934},
 {'title': 'One Night Stand', 'year': 1997},
 {'title': 'One Night with the King', 'year': 2006},
 {'title': 'One Night in City', 'year': 2007}]

### [Questão 2] Quantos filmes o diretor Quentin Tarantino dirigiu?

<!--
query = { 
         "directors": "Quentin Tarantino"
}

projection = { 
               "title": 1,
               "_id": 0   
}


result = db.movies.find(query, projection).distinct("title")
list(result)
-->

In [6]:
db.movies.find({"directors": "Quentin Tarantino"}).count()

14

### [Questão 3] Quantos filmes do gênero "Drama" o diretor Quentin Tarantino dirigiu entre 1990 e 2005?


<!--
query = { 
         "directors": "Quentin Tarantino",
         "genres" : "Drama",
           "$and" : [ 
                     { "year": {"$gte":1990}},
                     { "year": {"$gte":1990}}
                    ]
}

projection = { 
               "title": 1,
               "genres":1, 
               "year": 1,
               "_id": 0
}


result = db.movies.find(query, projection)
list(result)
-->

In [11]:
db.movies.find({ "$and": [
    { "year": { "$gte": 1990 } }, 
    { "year": { "$lte": 2005 } }, 
    {"directors": "Quentin Tarantino"},
    {"genres": "Drama"}
]}).count()

3

### [Questão 4] Qual filme do diretor Quentin Tarantino teve mais prêmios?

Dica: Sort em Pymongo https://www.w3schools.com/python/python_mongodb_sort.asp

<!-- 
query = { 
         "directors": "Quentin Tarantino"
}

projection = { 
               "title": 1,
               "awards.wins":1,
               "_id": 0   
}


result = db.movies.find(query, projection).sort("awards.wins",-1)
list(result)
-->

In [13]:
result2 = db.movies.find( {"directors": "Quentin Tarantino"},
                       {"_id": 0, 
                        "title": 1}
                       ).sort("awards.wins", -1).limit(1)
pprint(list(result2))

[{'title': 'Inglourious Basterds'}]


### [Questão Extra] Quais documentos (filmes) possuem mesmo nome e quantas vezes cada um se repete?

<!--

-->

In [15]:
pipeline = [
            {"$group": {"_id": "$title", "count": {"$sum": 1}}},
            {"$match": {"count":{"$gt":1}}},
            {"$sort":  {"count": -1}}
]


result = db.movies.aggregate(pipeline)
list(result)

[{'_id': 'The Terrorist', 'count': 6},
 {'_id': 'The Journey', 'count': 6},
 {'_id': 'Jane Eyre', 'count': 6},
 {'_id': 'The Hole', 'count': 6},
 {'_id': 'Hamlet', 'count': 6},
 {'_id': 'Peter Pan', 'count': 5},
 {'_id': 'The New Land', 'count': 5},
 {'_id': 'The Stranger', 'count': 5},
 {'_id': 'Love', 'count': 5},
 {'_id': 'Alice in Wonderland', 'count': 5},
 {'_id': 'First Love', 'count': 5},
 {'_id': 'The Emigrants', 'count': 5},
 {'_id': 'Madame Bovary', 'count': 5},
 {'_id': 'Mad As Hell', 'count': 5},
 {'_id': 'Les Misèrables', 'count': 5},
 {'_id': 'Creature', 'count': 5},
 {'_id': 'Macbeth', 'count': 5},
 {'_id': 'Wolf', 'count': 5},
 {'_id': 'The Hound of the Baskervilles', 'count': 4},
 {'_id': 'Carrie', 'count': 4},
 {'_id': 'Caravaggio', 'count': 4},
 {'_id': 'Cleopatra', 'count': 4},
 {'_id': 'Bang', 'count': 4},
 {'_id': 'The Inheritance', 'count': 4},
 {'_id': 'Red Dust', 'count': 4},
 {'_id': 'Thirst', 'count': 4},
 {'_id': 'The Phantom of the Opera', 'count': 4},
 {'_

### [Questão Extra] Quantos filmes cada diretor participou da direção? (ignore filmes repetidos)

In [14]:
pipeline = [
            {"$unwind": "$directors"},
            {"$group": {"_id": "$directors", "count": {"$sum": 1}}},
            {"$sort":  {"count": -1}}  
]

result = db.movies.aggregate(pipeline)
list(result)

[{'_id': 'Woody Allen', 'count': 40},
 {'_id': 'John Ford', 'count': 35},
 {'_id': 'John Huston', 'count': 34},
 {'_id': 'Takashi Miike', 'count': 34},
 {'_id': 'Werner Herzog', 'count': 33},
 {'_id': 'Martin Scorsese', 'count': 32},
 {'_id': 'Alfred Hitchcock', 'count': 31},
 {'_id': 'Sidney Lumet', 'count': 30},
 {'_id': 'George Cukor', 'count': 29},
 {'_id': 'Michael Apted', 'count': 29},
 {'_id': 'Steven Spielberg', 'count': 29},
 {'_id': 'Mario Monicelli', 'count': 29},
 {'_id': 'Steven Soderbergh', 'count': 28},
 {'_id': 'Spike Lee', 'count': 28},
 {'_id': 'Robert Altman', 'count': 28},
 {'_id': 'Clint Eastwood', 'count': 27},
 {'_id': 'Johnnie To', 'count': 27},
 {'_id': 'Jean-Luc Godard', 'count': 27},
 {'_id': 'Wim Wenders', 'count': 27},
 {'_id': 'Ken Loach', 'count': 27},
 {'_id': 'William Wyler', 'count': 26},
 {'_id': 'Michael Winterbottom', 'count': 26},
 {'_id': 'Ridley Scott', 'count': 25},
 {'_id': 'Ingmar Bergman', 'count': 25},
 {'_id': 'Brian De Palma', 'count': 24}

### [Questão Extra] Qual a somatória de prêmios recebidos pelos filmes para cada diretor? (ignore filmes repetidos)

In [16]:
pipeline = [
            {"$unwind": "$directors"},
            {"$group": {"_id": "$directors", "count": {"$sum": "$awards.wins"}}},
            {"$sort":  {"count": -1}}
    
]


result = db.movies.aggregate(pipeline)
list(result)

[{'_id': 'Steven Spielberg', 'count': 699},
 {'_id': 'Martin Scorsese', 'count': 587},
 {'_id': 'Alfonso Cuarèn', 'count': 577},
 {'_id': 'Peter Jackson', 'count': 527},
 {'_id': 'Ethan Coen', 'count': 496},
 {'_id': 'Joel Coen', 'count': 496},
 {'_id': 'Christopher Nolan', 'count': 488},
 {'_id': 'Quentin Tarantino', 'count': 445},
 {'_id': 'Ang Lee', 'count': 438},
 {'_id': 'Wes Anderson', 'count': 430},
 {'_id': 'David Fincher', 'count': 405},
 {'_id': 'Alejandro Gonzèlez Ièèrritu', 'count': 380},
 {'_id': 'Woody Allen', 'count': 378},
 {'_id': 'Steve McQueen', 'count': 367},
 {'_id': 'Clint Eastwood', 'count': 360},
 {'_id': 'Paul Thomas Anderson', 'count': 343},
 {'_id': 'Michael Haneke', 'count': 332},
 {'_id': 'Pedro Almodèvar', 'count': 328},
 {'_id': 'Alexander Payne', 'count': 326},
 {'_id': 'Kar Wai Wong', 'count': 292},
 {'_id': 'David O. Russell', 'count': 287},
 {'_id': 'James Cameron', 'count': 275},
 {'_id': 'Richard Linklater', 'count': 270},
 {'_id': 'Stephen Frears',